In [2]:
from bs4 import BeautifulSoup
import geopandas as gpd
import pandas as pd

In [19]:
import requests

## Get State Data

In [3]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [4]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Preventable Death Data

In [39]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36"
}

In [40]:
url = "https://apps.npr.org/dailygraphics/graphics/vax-preventable-deaths-20220510/"

In [41]:
r = requests.get(url, headers=headers)

In [42]:
soup = BeautifulSoup(r.text, "html.parser")

In [43]:
table = soup.find_all("table", {"id": "state-table"})[0]

In [45]:
preventable_deaths_dict = {
    "NAME": [],
    "preventable_deaths": [],
    "preventable_deaths_per_1m": [],
}
for tr in table.find_all("tr")[1:]:
    tds = tr.find_all("td")
    preventable_deaths_dict["NAME"].append(tds[0].text)
    preventable_deaths_dict["preventable_deaths"].append(
        int(tds[1].text.replace(",", ""))
    )
    preventable_deaths_dict["preventable_deaths_per_1m"].append(
        int(tds[2].text.replace(",", ""))
    )

In [46]:
preventable_deaths_df = pd.DataFrame(preventable_deaths_dict)

## Merge Data

In [50]:
preventable_deaths_gdf = states_df.merge(preventable_deaths_df, on="NAME", how="inner")

In [52]:
preventable_deaths_gdf = preventable_deaths_gdf.to_crs(9311)

In [53]:
preventable_deaths_gdf.to_file("data/preventable_deaths.gpkg")